In [1]:
import numpy as np

#starting_board = []

Första approach: två 8x8 plan på varandra
 + Läsbar, men
     - Man måste ändå hålla reda på att ena planet är förskjutet eftersom det representerar kryssen mellan rutorna
     - Detta plan har också en onödig rad och kolumn
 - Väldigt o-optimerad

In [2]:
test = np.zeros((2,8,8))
print(test)

[[[0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]]]


In [3]:
test[0]

array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])

För att komma fram till en mera optimal representation tror jag att jag först borde börja med att representare bägge "sidor" av planen skilt.

Från checkers implementationen som jag har lånat vet jag redan att den planen kan komprimeras till 4x8 eftersom pjäserna endast rör sig diagonalt



In [4]:
np.zeros((7,7))

array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.]])

In [5]:
# Lånat från checkers:
def expand(board):
	b = np.zeros((8, 8), dtype='b')
	for i in range(0, 8):
		if (i%2 == 0):
			b[i] = np.array([0, board[i*4], 0, board[i*4 + 1], 0, board[i*4 + 2], 0, board[i*4 + 3]])
		else:
			b[i] = np.array([board[i*4], 0, board[i*4 + 1], 0, board[i*4 + 2], 0, board[i*4 + 3], 0])
	return b

def compress(board):
	b = np.zeros((1,32), dtype='b')
	for i in range(0, 8):
		if (i%2 == 0):
			b[0, i*4 : i*4+4] = np.array([board[i, 1], board[i, 3], board[i, 5], board[i, 7]])
		else:
			b[0, i*4 : i*4+4] = np.array([board[i, 0], board[i, 2], board[i, 4], board[i, 6]])
	return b

In [6]:
import numpy as np
b = np.zeros((1, 8), dtype='b')
b[0,0] = 4

In [7]:
b[0,0]

4

## Kod för COIN spelarens tillåtna drag
Först vill jag ha en metod för att räkna ut vilka platser som blir diagonala "grannar" på det komprimerade brädet. Jag tror att den bästa approachen kommer att vara att köra denna metod en gång och spara resultatet i en list.

In [8]:
manual_diagonals = [[4,5],[5,6],[6,7],[7],[0,8],[0,1,8,9],[1,2,9,10],[2,3,10,11],[12,13,4,5],[13,14,5,6]]

In [9]:
def diagonal_steps(position):
    steps = []
    # Beacuse the checker board is compressed, diagonals work differently on add and even rows
    if (position // 4) % 2 == 0:
        if not ((position + 5) % 8 == 0):
            steps.append([position - 3])
        steps.append([position - 4])
        if not ((position + 5) % 8 == 0):
            steps.append([position + 5])
        steps.append([position + 4])
    else:
        if not ((position + 4) % 8 == 0):
            steps.append([position - 5])
        steps.append([position - 4])
        if not ((position + 4) % 8 == 0):
            steps.append([position + 3])
        steps.append([position + 4])
    i = 0
    while i < len(steps):
        if steps[i][0] < 0 or steps[i][0] > 31:
            steps.pop(i)
        else: i = i + 1
    steps.sort()
    return steps    

In [10]:
diagonals = []
for n in range(32):
    diagonals.append(diagonal_steps(n))

In [11]:
diagonals

[[[4], [5]],
 [[5], [6]],
 [[6], [7]],
 [[7]],
 [[0], [8]],
 [[0], [1], [8], [9]],
 [[1], [2], [9], [10]],
 [[2], [3], [10], [11]],
 [[4], [5], [12], [13]],
 [[5], [6], [13], [14]],
 [[6], [7], [14], [15]],
 [[7], [15]],
 [[8], [16]],
 [[8], [9], [16], [17]],
 [[9], [10], [17], [18]],
 [[10], [11], [18], [19]],
 [[12], [13], [20], [21]],
 [[13], [14], [21], [22]],
 [[14], [15], [22], [23]],
 [[15], [23]],
 [[16], [24]],
 [[16], [17], [24], [25]],
 [[17], [18], [25], [26]],
 [[18], [19], [26], [27]],
 [[20], [21], [28], [29]],
 [[21], [22], [29], [30]],
 [[22], [23], [30], [31]],
 [[23], [31]],
 [[24]],
 [[24], [25]],
 [[25], [26]],
 [[26], [27]]]

In [12]:
# BAD CODE! TODO: IGNORE
# Should be possible to find the right cross by index and move
def old_list_diagonal_crosses(diagonals):
    upwards = False
    i = len(diagonals) + 1
    while i < 49 + len(diagonals) + 1:
        for position in diagonals:
            for move in position:
                move.append(i)
                i = i + 1
                if i % 7 == 0:
                    upwards = not upwards
                    if upwards: i = i - 7
    return diagonals

In [13]:
def list_diagonal_crosses(diagonals):
    i = 33
    up = -1
    down = 3
    while i < 33 + 49:
        # each cross is next to 4 squares, what does each of them do?
        if i % 2 != 0:
            # odd = even
            # even crosses go up-right and down-left
            up += 1
            if (i-33) % 7 == 0:
                down += 1
        else:
            # odd crosses go up-left and down-right
            down +=1
            if (i-33) % 7 == 0:
                up += 1
        for move in diagonals[up]:
            if move[0] == down:
                move.append(i)
        for move in diagonals[down]:
            if move[0] == up:
                move.append(i)
        i += 1

In [14]:
list_diagonal_crosses(diagonals)

In [15]:
diagonals

[[[4, 33], [5, 34]],
 [[5, 35], [6, 36]],
 [[6, 37], [7, 38]],
 [[7, 39]],
 [[0, 33], [8, 40]],
 [[0, 34], [1, 35], [8, 41], [9, 42]],
 [[1, 36], [2, 37], [9, 43], [10, 44]],
 [[2, 38], [3, 39], [10, 45], [11, 46]],
 [[4, 40], [5, 41], [12, 47], [13, 48]],
 [[5, 42], [6, 43], [13, 49], [14, 50]],
 [[6, 44], [7, 45], [14, 51], [15, 52]],
 [[7, 46], [15, 53]],
 [[8, 47], [16, 54]],
 [[8, 48], [9, 49], [16, 55], [17, 56]],
 [[9, 50], [10, 51], [17, 57], [18, 58]],
 [[10, 52], [11, 53], [18, 59], [19, 60]],
 [[12, 54], [13, 55], [20, 61], [21, 62]],
 [[13, 56], [14, 57], [21, 63], [22, 64]],
 [[14, 58], [15, 59], [22, 65], [23, 66]],
 [[15, 60], [23, 67]],
 [[16, 61], [24, 68]],
 [[16, 62], [17, 63], [24, 69], [25, 70]],
 [[17, 64], [18, 65], [25, 71], [26, 72]],
 [[18, 66], [19, 67], [26, 73], [27, 74]],
 [[20, 68], [21, 69], [28, 75], [29, 76]],
 [[21, 70], [22, 71], [29, 77], [30, 78]],
 [[22, 72], [23, 73], [30, 79], [31, 80]],
 [[23, 74], [31, 81]],
 [[24, 75]],
 [[24, 76], [25, 77]],

In [16]:
manual_neighbors = [[1,7],[0,2,8]]

In [17]:
def get_neighbors(point, offset = 0):
    # offset is added to neigbors because the go board will be appended to the checkers board
    neighbors = []
    if not (point % 7 == 0):
        neighbors.append(point - 1 + offset)
    if not ((point + 1) % 7 == 0):
        neighbors.append(point + 1 + offset)
    neighbors.append(point - 7 + offset)
    neighbors.append(point + 7 + offset)
    i = 0
    while i < len(neighbors):
        if neighbors[i] - offset < 0 or neighbors[i] - offset > 48:
            neighbors.pop(i)
        else: i = i + 1
    return neighbors   
    

In [18]:
neighbors = []
for n in range(49):
    neighbors.append(get_neighbors(n, 0))

In [19]:
neighbors

[[1, 7],
 [0, 2, 8],
 [1, 3, 9],
 [2, 4, 10],
 [3, 5, 11],
 [4, 6, 12],
 [5, 13],
 [8, 0, 14],
 [7, 9, 1, 15],
 [8, 10, 2, 16],
 [9, 11, 3, 17],
 [10, 12, 4, 18],
 [11, 13, 5, 19],
 [12, 6, 20],
 [15, 7, 21],
 [14, 16, 8, 22],
 [15, 17, 9, 23],
 [16, 18, 10, 24],
 [17, 19, 11, 25],
 [18, 20, 12, 26],
 [19, 13, 27],
 [22, 14, 28],
 [21, 23, 15, 29],
 [22, 24, 16, 30],
 [23, 25, 17, 31],
 [24, 26, 18, 32],
 [25, 27, 19, 33],
 [26, 20, 34],
 [29, 21, 35],
 [28, 30, 22, 36],
 [29, 31, 23, 37],
 [30, 32, 24, 38],
 [31, 33, 25, 39],
 [32, 34, 26, 40],
 [33, 27, 41],
 [36, 28, 42],
 [35, 37, 29, 43],
 [36, 38, 30, 44],
 [37, 39, 31, 45],
 [38, 40, 32, 46],
 [39, 41, 33, 47],
 [40, 34, 48],
 [43, 35],
 [42, 44, 36],
 [43, 45, 37],
 [44, 46, 38],
 [45, 47, 39],
 [46, 48, 40],
 [47, 41]]

In [20]:
# Method for compressing the game board. I imagine it will mostly be used for human players
# but it can also be used to make the board at the start of the game
    
import numpy as np

def compress_board(stones, squares, grid):
    
    # Number of stones first
    board = [stones]
    
    # squares (unused squares never need to be represented)
    squares_list = squares.flatten().tolist()
    board = board + squares_list
    
    # Grid last
    grid_list = grid.flatten().tolist()
    board = board + grid_list
    
    return board

def decompress_board(board):
    stones = board[0]
    squares_list = board[1:33]
    squares = np.array(squares_list).reshape((8, 4))
    # add empty squares
    grid_list = board[33:]
    grid = np.array(grid_list).reshape((7, 7))
    return stones, squares, grid

def create_starting_board():
    squares = np.zeros((8,4), dtype='b')
    squares[3:5,1:3] = 1
    squares[2,1] = 1
    squares[5,2] = 1
    grid = np.zeros((7,7), dtype='b')
    starting_board = compress_board(66,squares,grid)
    return starting_board

# IMPORTANT! Make sure the positions lists are always up to date
def list_checker_positions(board):
    checker_list = []
    for index, square in enumerate(board[1:33]):
        if square == 1:
            checker_list.append(index + 1)
    return checker_list

In [75]:
def draw_board(board):
    stones, squares, grid = decompress_board(board)
    print("\n")
    for i, row in enumerate(squares):
        #print(i, row)
        if i%2 == 0:
            for j, square in enumerate(row):
                print('0', square, end=' ')
            print('')
            if i<7:
                print(grid[i, :])
        else:
            for j, square in enumerate(row):
                print(square, '0', end=' ')
            print('')
            if i<7:
                print(grid[i, :])

In [22]:
starting_board = create_starting_board()
checker_positions = list_checker_positions(starting_board)

In [23]:
draw_board(starting_board)

0 0 0 0 0 0 0 0 
[0 0 0 0 0 0 0]
0 0 0 0 0 0 0 0 
[0 0 0 0 0 0 0]
0 0 0 1 0 0 0 0 
[0 0 0 0 0 0 0]
0 0 1 0 1 0 0 0 
[0 0 0 0 0 0 0]
0 0 0 1 0 1 0 0 
[0 0 0 0 0 0 0]
0 0 0 0 1 0 0 0 
[0 0 0 0 0 0 0]
0 0 0 0 0 0 0 0 
[0 0 0 0 0 0 0]
0 0 0 0 0 0 0 0 


In [24]:
checker_positions

[10, 14, 15, 18, 19, 23]

In [25]:
starting_board[0]

66

In [26]:
decompress_board(starting_board)

(66,
 array([[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 1, 1, 0],
        [0, 1, 1, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]]),
 array([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]]))

In [27]:
import copy
# TODO: Get all free crosses next to stones
# TODO: Check for one free cross next to that stone
# TODO: Make a special case for the first move of the game

def get_guerila_moves(board):
    move_list=[]
    # TODO: Find out if it's necessary to make a copy of the board for each move
    # First move, if the guerilla player is still holding all their stones
    if board[0] == 66:
        for i in range(48):
            if i < 42:
                new_board = copy.copy(board)
                new_board[i + 33] = 1
                new_board[i + 33 + 7] = 1
                move_list.append(new_board)
            if (i+1)%7 != 0:
                new_board = copy.copy(board)
                new_board[i + 33] = 1
                new_board[i + 33 + 1] = 1
                move_list.append(new_board)
    else:
        # Find occupied crosses
        for index, cross in enumerate(board[33:]):
            if cross == 1:
                # TODO: Make sure the name and index of the neighbors list is correct!
                for neighbor in neighbors[index]:
                    if board[neighbor + 33] == 0:
                        for neighborbor in neighbors[neighbor]:
                            if board[neighborbor] == 0:
                                new_board = copy.copy(board)
                                new_board[neighbor + 33] = 1
                                new_board[neighborbor + 33] = 1
                                # TODO: Write check_surround
                                new_board = check_surround(new_board)
                                move_list.append(new_board)
                                # TODO: Find out if it's possibe to avoid all duplicate moves!
    # TODO: Check if any COIN checkers have been surrounded
    return move_list

def check_surround(board, positions = checker_positions):
    for position in positions:
        captured = True
        # TODO: Make sure "diagonal_crosses" names the list
        for cross in diagonal_crosses[position]:
            if cross[1] == 0:
                captured = False
        if captured:
            board[position] = 0
    return board

För att få alla möjliga drag gäller det att kolla var det finns brickor. 
 + Sådana moves som leder ovanpå en annan bricka måste uteslutas
 + För de moves som går över motståndarens sten måste en till (obligatorisk) move hämtas
     + OBS att detta gäller endast om det finns en till move som passerar över motståndarens sten

In [100]:
# TODO: Get moves for all existing COIN pieces, removing collisions
# TODO: Make a list of which cross to check on a particular COIN move
import copy

def get_COIN_moves(board):
    move_list=[]
    
    # Find occupied squares
    # for index, square in board[1:33]:
    #    if square == 1:
    
    # UNNESESSARY! i'VE INVENTED THE checker_positions LIST!!!
    
    for position in checker_positions:
            # Check for possible moves
            #TODO: CHECK diagonals LIST!
            for diagonal in diagonals[position - 1]:
                if board[diagonal[0] + 1] == 0:
                    new_board = copy.copy(board)
                    new_board[position] = 0
                    new_board[diagonal[0] + 1] = 1
                    # This conditional isn't triggered when it should be
                    if board[diagonal[1]] == 1:
                        new_board[diagonal[1]] = 0
                        move_list += capture_and_move(new_board, diagonal[0] + 1)
                    else:
                        move_list.append(new_board)
                        # Capture and recurse
            #TODO: Check if diagonal is occupied
            #TODO: Check if a move is a capture
            # TODO: MORE!
    #TODO: Return a list of possible moves
    # A move is represented by the board state produed by that move
    return move_list

def capture_and_move(board, position):
    new_moves = []
    jumped = False
    for diagonal in diagonals[position - 1]:
        if board[diagonal[1]] == 1 and board[diagonal[0] + 1] == 0:
            jumped = True
            new_board = copy.copy(board)
            new_board[position] = 0
            new_board[diagonal[0] + 1] = 1
            new_board[diagonal[1]] = 0
            new_moves += capture_and_move(new_board, diagonal[0] + 1)
    if not jumped:
        new_moves.append(board)
    return new_moves
    # BROBLEM?: The possible series of captures will be both recursive and branching

In [29]:
# TODO: Make a test for the code in the cell above

first_moves = get_guerila_moves(starting_board)

In [30]:
len(first_moves)

84

In [31]:
draw_board(first_moves[4])
# TODO: FIX GUERILLA MOVES

0 0 0 0 0 0 0 0 
[0 0 1 0 0 0 0]
0 0 0 0 0 0 0 0 
[0 0 1 0 0 0 0]
0 0 0 1 0 0 0 0 
[0 0 0 0 0 0 0]
0 0 1 0 1 0 0 0 
[0 0 0 0 0 0 0]
0 0 0 1 0 1 0 0 
[0 0 0 0 0 0 0]
0 0 0 0 1 0 0 0 
[0 0 0 0 0 0 0]
0 0 0 0 0 0 0 0 
[0 0 0 0 0 0 0]
0 0 0 0 0 0 0 0 


In [101]:
first_COIN_moves = get_COIN_moves(first_moves[4])

In [44]:
first_moves[4][35]

1

In [45]:
diagonals[10]

[[6, 44], [7, 45], [14, 51], [15, 52]]

In [103]:
draw_board(first_COIN_moves[1])



0 0 0 0 0 0 0 0 
[0 0 1 0 0 0 0]
0 0 0 0 1 0 0 0 
[0 0 1 0 0 0 0]
0 0 0 0 0 0 0 0 
[0 0 0 0 0 0 0]
0 0 1 0 1 0 0 0 
[0 0 0 0 0 0 0]
0 0 0 1 0 1 0 0 
[0 0 0 0 0 0 0]
0 0 0 0 1 0 0 0 
[0 0 0 0 0 0 0]
0 0 0 0 0 0 0 0 
[0 0 0 0 0 0 0]
0 0 0 0 0 0 0 0 


Det ser ut som att get_COIN_moves inte har upptäckt stenarna. Är det fel i diagonals?

In [102]:
len(first_COIN_moves)

10

In [99]:
for board in first_COIN_moves:
    draw_board(board)



0 0 0 1 0 0 0 0 
[0 0 0 0 0 0 0]
0 0 0 0 0 0 0 0 
[0 0 0 0 0 0 0]
0 0 0 0 0 0 0 0 
[0 0 0 0 0 0 0]
0 0 1 0 1 0 0 0 
[0 0 0 0 0 0 0]
0 0 0 1 0 1 0 0 
[0 0 0 0 0 0 0]
0 0 0 0 1 0 0 0 
[0 0 0 0 0 0 0]
0 0 0 0 0 0 0 0 
[0 0 0 0 0 0 0]
0 0 0 0 0 0 0 0 


0 0 0 0 0 0 0 0 
[0 0 1 0 0 0 0]
0 0 1 0 0 0 0 0 
[0 0 0 0 0 0 0]
0 0 0 0 0 0 0 0 
[0 0 0 0 0 0 0]
0 0 1 0 1 0 0 0 
[0 0 0 0 0 0 0]
0 0 0 1 0 1 0 0 
[0 0 0 0 0 0 0]
0 0 0 0 1 0 0 0 
[0 0 0 0 0 0 0]
0 0 0 0 0 0 0 0 
[0 0 0 0 0 0 0]
0 0 0 0 0 0 0 0 


0 0 0 0 0 0 0 0 
[0 0 1 0 0 0 0]
0 0 0 0 1 0 0 0 
[0 0 1 0 0 0 0]
0 0 0 0 0 0 0 0 
[0 0 0 0 0 0 0]
0 0 1 0 1 0 0 0 
[0 0 0 0 0 0 0]
0 0 0 1 0 1 0 0 
[0 0 0 0 0 0 0]
0 0 0 0 1 0 0 0 
[0 0 0 0 0 0 0]
0 0 0 0 0 0 0 0 
[0 0 0 0 0 0 0]
0 0 0 0 0 0 0 0 


0 0 0 0 0 0 0 0 
[0 0 1 0 0 0 0]
0 0 0 0 0 0 0 0 
[0 0 1 0 0 0 0]
0 1 0 1 0 0 0 0 
[0 0 0 0 0 0 0]
0 0 0 0 1 0 0 0 
[0 0 0 0 0 0 0]
0 0 0 1 0 1 0 0 
[0 0 0 0 0 0 0]
0 0 0 0 1 0 0 0 
[0 0 0 0 0 0 0]
0 0 0 0 0 0 0 0 
[0 0 0 0 0 0 0]
0 0 0 0 0 0 0 0 


In [36]:
diagonals[10]

[[6, 44], [7, 45], [14, 51], [15, 52]]

In [37]:
first_COIN_moves[0][48]

0

In [38]:
len(starting_board)

82

In [39]:
test_board = list(range(82))

In [40]:
decompress_board(test_board)

(0,
 array([[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12],
        [13, 14, 15, 16],
        [17, 18, 19, 20],
        [21, 22, 23, 24],
        [25, 26, 27, 28],
        [29, 30, 31, 32]]),
 array([[33, 34, 35, 36, 37, 38, 39],
        [40, 41, 42, 43, 44, 45, 46],
        [47, 48, 49, 50, 51, 52, 53],
        [54, 55, 56, 57, 58, 59, 60],
        [61, 62, 63, 64, 65, 66, 67],
        [68, 69, 70, 71, 72, 73, 74],
        [75, 76, 77, 78, 79, 80, 81]]))

In [63]:
checker_positions

[10, 14, 15, 18, 19, 23]

In [66]:
starting_board[23]

1